In [1]:
import ssl
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import time
import re
import statistics
import matplotlib.pyplot as py
import pandas as pd 
import ...

In [2]:
def get_links(pages = 1):
    #gets the links from trulia properties page 
    # if length of list is zero, captcha blocked request so use a new VPN location 
    #pages: number of pages of trula to scrape
    #returns: list of links
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    i = 0
    property_links = []
    while i < pages:
        url = 'https://www.trulia.com/NY/New_York/' + "%s_p/"%str(i)
        req = Request(url, headers={"User-Agent": 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage , 'html.parser')
        for link in soup.findAll('a'):
            l = str(link.get('href'))
            if sum(map(str.isdigit, l[-17:])) == 15: property_links.append("https://www.trulia.com" + l)
        property_links = property_links[::2]
        print("finishing loop" + " " + str(i+1))
        i += 1
        time.sleep(5)
        
    return(property_links)

In [28]:
links = get_links(pages = 5)
feature_dict_list = {}
all_features = []
all_links = []

finishing loop 1
finishing loop 2
finishing loop 3
finishing loop 4
finishing loop 5


In [38]:
for index,url in enumerate(links[30:40]): #iterates through all URLs from the get_links func
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()  
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    feature_dict = {}

    for sou in soup.findAll('span', attrs={"class":"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"}):
        clean_data = str(sou).split("gmLKqq\">")
        clean_data = clean_data[1].split("</span>")[0]
        dict_info = clean_data.split(':')
        if len(dict_info)>1:
            feature_dict[dict_info[0]] = dict_info[1]

    all_features.append(feature_dict);all_links.append(url)



In [45]:
df = pd.DataFrame.from_dict(all_features)
pd.set_option('display.max_columns', None)
df

,Stories,Cooling System,Days on Market,Year Built,Property Type,Price Per Sqft,Units in Building,HOA Fee,Lot Area,Number of Rooms,Number of Bedrooms,Number of Bathrooms,Number of Bathrooms (full),Living Area,Utilities,Appliances,Heating,Air Conditioning,Heating Fuel,Flooring,Common Walls,Floors,Parking,Sewer,Property Subtype,Architecture,Construction Materials,Included in Sale,List Price,MLS Status,City,Community,Elementary School,Elementary School District,Jr High / Middle School,Jr High / Middle School District,High School,High School District,MLS/Source ID,Community Features,HOA Fee Includes,Ownership,Year Updated,Basement,Number of Stories,Exterior,Attic,Types of Rooms,Electric,Levels,Condition,Parcel Number,Building Area,Listing Agreement Type,Listing Url,Water View,"<a href=""https",Accessibility,Patio / Porch,Structure Type,Building Name,Building,HOA Name,HOA Phone,Special Conditions,Number of Bathrooms (three quarters),Foundation Area,Roof,Foundation,Parking Spaces,Number Of Vacant Units,Laundry,Number of Bathrooms (half),Number of Garage Spaces,Fencing,Pool
0,12,Central,&lt;1 Day on Trulia,1912,Coop,$81,48,"$7,036/Monthly",7662 sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,14,1925,Residential,$145,NaN,$483/Monthly,NaN,3,1,1,1,550 Square Feet,Public Garbage Removal,"Microwave, Oven, Refrigerator, Stainless Stee...","Oil,Hot Water",Window Unit(s),Oil,Hardwood,2+ Common Walls,Hardwood,On Street,Public Sewer,Stock Cooperative,Garden Apartment,Brick,"A/C Units, Front Gate, Microwave, Refrigerato...","$80,000",A,BRONX,1021 Avenue Saint John,Call Listing Agent,City of New York,Call Listing Agent,City of New York,Call Listing Agent,City of New York,H6220496,Park,"Heat, Hot Water",Co-Op,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,&lt;1 Day on Trulia,1942,Coop,$469,108,$684/Monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Central,&lt;1 Day on Trulia,1920,Multi Family,$433,3,NaN,2000 sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,&lt;1 Day on Trulia,1985,Residential,"$1,308",NaN,NaN,NaN,7,4,3,3,1900 Square Feet,Other Garbage Removal,Dishwasher,"Other,See Remarks",None,Other,NaN,NaN,NaN,Other,Other,Condominium,NaN,NaN,Dishwasher,"$2,485,000",A,New York,Bel Canto Condominium,Call Listing Agent,City of New York,Call Listing Agent,City of New York,NaN,City of New York,H6222404,NaN,NaN,Condo,NaN,See Remarks,26,Balcony,See Remarks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,&lt;1 Day on Trulia,1926,Coop,$974,186,"$2,522/Monthly",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,NaN,&lt;1 Day on Trulia,2007,Multi Family,$247,2,NaN,3031 sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Colonial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11123779,NaN,NaN,NaN,NaN,NaN,NaN,Brick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Central,&lt;1 Day

In [ ]:
for house in all_features:
    for f in house:
        dict_key,dict_value = f.split(':')

In [62]:
# For ignoring SSL certificate errors  (reused code)

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [63]:
#url = input("Enter URL: ")
url = 'https://www.trulia.com/p/ny/staten-island/365-victory-blvd-staten-island-ny-10301--2008726710'

In [28]:
# Use Mozilla access agent

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

In [29]:
# Creating a BeautifulSoup object of the html page for easy extraction of data.

soup = BeautifulSoup(webpage, 'html.parser')
html = soup.prettify('utf-8')
property_info = {}

In [30]:
# This code block will get you a one liner description of the listed property

for meta in soup.findAll('meta', attrs={'name': 'description'}):
    try:
        property_info['description'] = meta['content']
        break
    except: 
        pass

property_info['description']

KeyError: 'description'

In [31]:
#Store url 

property_info['link'] = url

property_info['link']

'https://www.trulia.com/p/ny/new-york/309-e-houston-st-1a-new-york-ny-10002--2478352572'

In [33]:
#Get beds, baths, floorspace

for bed_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bedrooms"}):
    number_beds = re.findall(r"[0-9]+\s[bB]eds*", str(bed_html))
    if number_beds:
        property_info["beds"] = int(number_beds[0].split(" ")[0])

for bath_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bathrooms"}):
    number_baths = re.findall(r"[0-9]+\s[bB]aths*", str(bath_html))
    if number_baths:
        property_info["baths"] = int(number_baths[0].split(" ")[0])

for floor_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-floorspace"}):
    floor_footage = re.findall(r"[0-9]+,[0-9]+\ssqft*", str(floor_html))
    property_info["floor_space"] = {}
    if floor_footage:
        property_info["floor_space"]["value"] = int(floor_footage[0].split(" ")[0].replace(",", ""))
        property_info["floor_space"]["unit"] = floor_footage[0].split(" ")[1]

property_info


{'link': 'https://www.trulia.com/p/ny/new-york/309-e-houston-st-1a-new-york-ny-10002--2478352572'}

In [69]:
#Extracts features for given house
for index, sou in enumerate(soup.findAll('span', attrs={"class":"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"})):
    clean_data = str(sou).split("gmLKqq\">")
    clean_data = clean_data[1].split("</span>")[0]
clean_data

'Lot Area: 3001 sqft'